In [ ]:
%%capture
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import cv2
import PIL

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm


import matplotlib.pyplot as plt

import tqdm.notebook as tqdm
import copy
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import os
import random
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
# Model Paths
mobile_net_path = '../input/mobilenetv3trained/BestValLoss.pth'
eff_net_fold_0_path = "../input/efficientnetfolds/fold0/fold_0_F.pth"
eff_net_fold_1_path = "../input/efficientnetfolds/fold1/fold_0_F.pth"
eff_net_fold_2_path = "../input/efficientnetfolds/fold2/fold_0_F.pth" # Trained Model Paths

In [ ]:
# HYPERPARAMETERS
dataframe_path = "../input/plant-pathology-2021-fgvc8/train.csv"
test_df_path = '../input/plant-pathology-2021-fgvc8/sample_submission.csv'
test_df = pd.read_csv(test_df_path)
test_df = test_df.set_index('image')
BASE_PATH = "../input/plant-pathology-2021-fgvc8/test_images/"
TEST_BATCH_SIZE = 32
INPUT_WIDTH = 320
IMAGE_SIZE = 320
INPUT_HEIGHT = int(INPUT_WIDTH *1.5)

In [ ]:
test_transforms = A.Compose([
    A.Resize(INPUT_WIDTH, INPUT_HEIGHT),
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
dataFrame = pd.read_csv(dataframe_path)
dataFrame = dataFrame.set_index("image")
def process_dataframe(dataframe):
    '''
    dataFrame: pandas dataframe containing image ids and labels
    '''
    classes = []
    count = 0
    for row in dataframe.iterrows():
        labels = str.split(row[1][0])
        classes += labels
    classes = sorted(list(set(classes)))
    classes.remove('healthy')
    return classes # Multi Class Classification Over 5 Classes
CLASSES = process_dataframe(dataFrame)

In [ ]:
def add_base(df, base_path):
    '''
    Adds base paths to every entry in the df, so no more need for base_path
    '''
    pred = {'image': [], 'labels': []}
    for row in df.iterrows():
        img = base_path + row[0]
        
        labels = row[1][0]
        pred['image'] += [img]
        pred['labels'] += [labels]
    df = pd.DataFrame(pred)
    df = df.set_index('image')
    return df

In [ ]:
processed_test_df = add_base(test_df, BASE_PATH)

Process and Load Test Images

In [ ]:
class TestPlantDataset(torch.utils.data.Dataset):
    def __init__(self, test_df, transforms):
        self.test_df = test_df
        self.indices = self.test_df.index.values
        self.transforms = transforms
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        index = self.indices[idx]
        # Extract index from file_path
        image_id =""
        for idx in range(len(index) - 1, -1, -1):
            if index[idx] == "/":
                image_id = index[idx + 1:]
                break
        # Load image in 
        image = cv2.imread(index)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = self.transforms(image = image)['image']
        return image, image_id

In [ ]:
# Load Datasets for Initial Pred
test_dataset = TestPlantDataset(processed_test_df, test_transforms)
# Load in Dataloader
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = TEST_BATCH_SIZE, shuffle = False, worker_init_fn = seed_worker)

# Model Blocks

In [ ]:
class StridedConvBlock(nn.Module):
    def __init__(self, in_features, out_features, kernel_size, padding, groups, stride):
        super().__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size = kernel_size, padding = padding, groups = groups, stride = stride)
        self.bn = nn.BatchNorm2d(out_features)
        self.act1 = nn.SiLU(inplace = True)
    def forward(self, x):
        return self.bn(self.act1(self.conv(x)))

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_features, out_features, kernel_size, padding, groups):
        super().__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size = kernel_size, padding = padding, groups = groups) 
        self.bn = nn.BatchNorm2d(out_features)
        self.act1 = nn.SiLU(inplace = True)
    def forward(self, x):
        return self.bn(self.act1(self.conv(x)))

In [ ]:
class SqueezeExcite(nn.Module):
    def __init__(self, in_dim, inner_dim):
        super().__init__()
        self.in_dim = in_dim
        self.inner_dim = inner_dim
        
        self.Squeeze = nn.Linear(self.in_dim, self.inner_dim)
        self.Excite = nn.Linear(self.inner_dim, self.in_dim)
        self.act1 = nn.SiLU(inplace = True) 
    def forward(self, x):
        max_pool, _ = torch.max(x, dim = -1) 
        max_pool, _ = torch.max(max_pool, dim = -1)
        
        squeezed = self.act1(self.Squeeze(max_pool))
        excite = torch.sigmoid(self.Excite(squeezed)).unsqueeze(-1).unsqueeze(-1)
        return excite * x

In [ ]:
class BottleNeck(nn.Module):
    def __init__(self, in_features, inner_features, device, stochastic_depth = 0.2):
        super().__init__()
        self.stochastic_depth = stochastic_depth
        self.in_features = in_features
        self.inner_features = inner_features
        self.device = device
        
        self.Squeeze = ConvBlock(self.in_features, self.inner_features, 1, 0, 1)
        self.Process = ConvBlock(self.inner_features, self.inner_features, 3, 1, 1)
        self.Expand = ConvBlock(self.inner_features, self.in_features, 1, 0, 1)
        self.SE = SqueezeExcite(self.in_features, self.in_features // 16)
       
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        if self.training and random.random() <= self.stochastic_depth:
            return x
        squeeze = self.Squeeze(x)
        process = self.Process(squeeze)
        expand = self.Expand(process)
        excited = self.SE(expand)
        return self.gamma * excited + x

In [ ]:
class InvertedBottleNeck(nn.Module):
    def __init__(self, in_dim, inner_dim, device, stochastic_depth = 0.2):
        super().__init__()
        self.stochastic_depth = stochastic_depth
        self.device = device
        self.in_dim = in_dim
        self.inner_dim = inner_dim
        
        self.expand = ConvBlock(self.in_dim, self.inner_dim, 1, 0, 1)
        self.depthwise = ConvBlock(self.inner_dim, self.inner_dim, 3, 1, self.inner_dim)
        self.SE = SqueezeExcite(self.inner_dim, self.inner_dim // 16)
        self.squeeze = ConvBlock(self.inner_dim, self.in_dim, 1, 0, 1)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        if self.training and random.random() <= self.stochastic_depth:
            return x 
        expanded = self.expand(x)
        depthwise = self.depthwise(expanded)
        excited = self.SE(depthwise)
        squeezed = self.squeeze(excited)
        return self.gamma * squeezed + x

In [ ]:
class CNNDownSampler(nn.Module):
    '''
    Uses Convolution and Resizing to Downsample images
    '''
    def __init__(self, out_size, device):
        super().__init__()
        self.device = device
        self.out_size = out_size
        self.initialCNN = nn.Sequential(*[
            ConvBlock(3, 16, 1, 0, 1), ConvBlock(16, 16, 1, 0, 1)])
        
        self.post_processing = nn.Sequential(*[
            ConvBlock(16, 16, 1, 0, 1) for i in range(2) 
        ])
        self.proj = ConvBlock(16, 3, 1, 0, 1)
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        resized = F.interpolate(x, size = (self.out_size, self.out_size), mode = 'bilinear')
        initial_processing = self.initialCNN(x)
        # Resize
        resize_processed = F.interpolate(initial_processing, size = (self.out_size, self.out_size), mode = 'bilinear')
        # Further Processing
        further_processed = self.post_processing(resize_processed)
        proj = self.proj(further_processed)
        return proj * self.gamma + resized

In [ ]:
class MobileNetv3(nn.Module):
    '''
    BaseLine MobileNet v3 Based Model
    '''
    def freeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = False
    def __init__(self, out_size, num_classes, device, dropout = 0.2, stochastic_depth = 0.2):
        super().__init__()
        self.drop_prob = dropout
        self.stochastic_depth = stochastic_depth
        self.model_name = 'mobilenetv3_large_100'
        self.model = timm.create_model(self.model_name, pretrained = False)
        
        self.out_size = out_size
        self.num_classes = num_classes
        self.device = device
        self.downsampler = CNNDownSampler(self.out_size, self.device) 
        
        self.conv1 = self.model.conv_stem
        self.bn1 = self.model.bn1
        self.act1 = self.model.act1
        
        self.block0 = self.model.blocks[0] # (16)
        self.block1 = self.model.blocks[1] # (24)
        self.block2 = self.model.blocks[2] 
        self.block3 = self.model.blocks[3]
        self.block4 = self.model.blocks[4]
        self.block5 = self.model.blocks[5]
        self.block6 = self.model.blocks[6] # We won't be using this block, it's very expensive
        
        # Freeze Initial Layers
        #self.freeze(self.conv1)
        #self.freeze(self.bn1)
        #self.freeze(self.block0)
        #self.freeze(self.block1)
        #self.freeze(self.block2)
        #self.freeze(self.block3)
        #self.freeze(self.block4)
        #self.freeze(self.block5)
        
        # Custom Layers
        self.Attention1 = SqueezeExcite(40, 16)
        self.Attention2 = SqueezeExcite(112, 32)
        self.Attention3 = SqueezeExcite(160, 48)
        self.layer4 = nn.Sequential(*[
            ConvBlock(160, 320, 1, 0, 1)
        ] + [
            BottleNeck(320, 64, self.device, stochastic_depth = self.stochastic_depth) for i in range(5)
        ])
        
        self.Attention4 = SqueezeExcite(320, 96)
        self.layer5 = nn.Sequential(*[
            StridedConvBlock(320, 512, 1, 0, 1, 2)
        ] + [
            BottleNeck(512, 128, self.device, stochastic_depth = self.stochastic_depth) for i in range(3)
        ])
        
        self.global_avg = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(self.drop_prob)
        self.Linear = nn.Linear(512, self.num_classes)
    def forward(self, x):
        downsampled = self.downsampler(x)
        features0 = self.bn1(self.act1(self.conv1(downsampled)))
        block0 = self.block0(features0) # (16, 160)
        block1 = self.block1(block0) # (24, 80)
        block2 = self.block2(block1) # (40, 40)
        block2 = self.Attention1(block2)
        
        block3 = self.block3(block2) # (80, 20)
        block4 = self.block4(block3) # (112, 20)
        block4 = self.Attention2(block4)
        
        block5 = self.block5(block4) # (160, 10)
        block5 = self.Attention3(block5)
        
        layer4 = self.layer4(block5) 
        layer4 = self.Attention4(layer4) # (320, 10)
        
        layer5 = self.layer5(layer4) # (512, 5) 
        
        avg_pool = torch.squeeze(self.global_avg(layer5))
        avg_pool = self.dropout(avg_pool)
        return self.Linear(avg_pool)

In [ ]:
class CustomLoss(nn.Module):
    '''
    Separates BCE loss into 0s and 1 to weight them evenly(1s are sparse)
    '''
    def __init__(self):
        super().__init__()
        self.criterion = nn.BCEWithLogitsLoss()
    def forward(self, pred, y_true):
        one_bools = y_true == 1
        zero_bools = y_true == 0
        pred_one = pred[one_bools]
        pred_zeros = pred[zero_bools]
        
        one_loss = self.criterion(pred_one, torch.ones_like(pred_one, device = pred_one.device))
        zero_loss = self.criterion(pred_zeros, torch.zeros_like(pred_zeros, device = pred_zeros.device))
        
        return one_loss + zero_loss

In [ ]:
class MobileSolverQTPi(nn.Module):
    def __init__(self, num_classes, device):
        super().__init__()
        self.out_size = IMAGE_SIZE
        self.num_classes = num_classes
        self.device = device
        self.dropout = 0.5
        self.stochastic_depth = 0.0
        self.model = MobileNetv3(self.out_size, self.num_classes, self.device, dropout = self.dropout, stochastic_depth = self.stochastic_depth)
    def forward(self, x):
        self.eval()
        with torch.no_grad():
            return torch.sigmoid(self.model(x))

In [ ]:
class ModifiedEffNetQT(nn.Module):
    '''
    EfficientNet B4 Variant of the Same Model
    '''
    def freeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = False
    def unfreeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = True
    def __init__(self, out_size, num_classes, device, dropout = 0.2, stochastic_depth = 0.2):
        super().__init__()
        self.model_name = "tf_efficientnet_b4_ns"
        self.model = timm.create_model(self.model_name, pretrained = False)
        
        self.out_size = out_size
        self.num_classes = num_classes
        self.device = device
        self.drop_prob = dropout
        self.stochastic_depth = stochastic_depth
        
        self.downsampler = CNNDownSampler(self.out_size, self.device)
        
        self.conv1 = self.model.conv_stem
        self.bn1 = self.model.bn1
        self.act1 = self.model.act1
    
        self.block0 = self.model.blocks[0]
        self.block1 = self.model.blocks[1]
        self.block2 = self.model.blocks[2]
        self.block3 = self.model.blocks[3]
        self.block4 = self.model.blocks[4]
        self.block5 = self.model.blocks[5]
        self.block6 = self.model.blocks[6]
       
        # Freeze Values(Uncomment if you replace self.downsampler with nn.Identity)
        #self.freeze(self.conv1)
        #self.freeze(self.bn1)
        #self.freeze(self.block0)
        #self.freeze(self.block1)
        #self.freeze(self.block2)
        
        self.Attention1 = SqueezeExcite(56, 16)
        self.Attention2 = SqueezeExcite(160, 32)
        self.Attention3 = SqueezeExcite(448, 128)
        
        self.layer5 = nn.Sequential(*[
            StridedConvBlock(448, 768, 1, 0, 1, 2)
        ] + [
           InvertedBottleNeck(768, 1536, self.device, stochastic_depth = self.stochastic_depth) for i in range(3)
        ])
        self.proj = ConvBlock(768, 2048, 1, 0, 1)
        
        self.global_avg = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(self.drop_prob)
        self.Linear = nn.Linear(2048, self.num_classes)
    def forward(self, x):
        downsampled = self.downsampler(x)
        features0 = self.bn1(self.act1(self.conv1(downsampled))) # (B, 48, 160, 160)
        
        block0 = self.block0(features0)
        block1 = self.block1(block0)
        block2 = self.block2(block1) # (B, 56, 40, 40)
        # Attention1
        block2 = self.Attention1(block2)
        
        block3 = self.block3(block2)
        block4 = self.block4(block3) # (B, 160, 20, 20)
        # Attention2
        block4 = self.Attention2(block4)
        
        block5 = self.block5(block4)
        block6 = self.block6(block5)
        # attention 3
        block6 = self.Attention3(block6) # (B, 448, 10, 10)
        
        layer5 = self.layer5(block6)
        layer5 = self.proj(layer5)
        global_avg = torch.squeeze(self.global_avg(layer5))
        dropped = self.dropout(global_avg)
        return self.Linear(dropped)
        

In [ ]:
class ModifiedResNetQT(nn.Module):
    '''
    Modified ResNet200D
    '''
    def freeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = False
    def unfreeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = True
    def __init__(self, out_size, num_classes, device, dropout = 0.2, stochastic_depth = 0.2):
        super().__init__()
        self.device = device
        self.num_classes = num_classes
        self.dropout = dropout 
        self.out_size = out_size
        self.device = device
        self.stochastic_depth = stochastic_depth
        self.model_name = 'resnet200d'
        self.model = timm.create_model(self.model_name, pretrained = False)
        
        self.downsampler = CNNDownSampler(self.out_size, self.device) 
        
        self.conv1 = self.model.conv1
        self.bn1 = self.model.bn1
        self.act1 = self.model.act1
        self.pool = self.model.maxpool
        
        self.layer1 = self.model.layer1
        self.layer2 = self.model.layer2
        self.layer3 = self.model.layer3
        self.layer4 = self.model.layer4 # we won't use this layer.
        
        # Freeze Initial Layers
        #self.freeze(self.conv1)
        #self.freeze(self.bn1)
        #self.freeze(self.layer1)
        #self.freeze(self.layer2)
        
        # Custom Layer
        self.Attention2 = SqueezeExcite(512, 128)
        self.Attention3 = SqueezeExcite(1024, 256)
        self.Attention4 = SqueezeExcite(2048, 512)
        
        self.layer4 = nn.Sequential(*[
            StridedConvBlock(1024, 1536, 1, 0, 1, 2)
        ] + [
            BottleNeck(1536, 512, self.device, stochastic_depth = self.device) for i in range(5)
        ])
        self.layer5 = nn.Sequential(*[
            StridedConvBlock(1536, 2048, 1, 0, 1)
        ] + [
            BottleNeck(2048, 512, self.device, stochastic_depth = self.device) for i in range(2)
        ])
        self.global_avg = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(self.drop_prob)
        self.Linear = nn.Linear(2048, self.num_classes)
    def forward(self, x):
        '''
        x: Tensor(B, 3, H, W)
        '''
        downsampled = self.downsampler(x) # (B, 3, out_size, out_size)
        features0 = self.pool(self.bn1(self.act1(self.conv1(downsampled)))) # (B, 64, 160, 160)
        
        layer1 = self.layer1(features0)
        layer2 = self.layer2(layer1)
        # Attention2
        layer2 = self.Attention2(layer2)
        
        layer3 = self.layer3(layer2)
        # Attention 3
        layer3 = self.Attention3(layer3)
        
        layer4 = self.layer4(layer4)
        layer4 = self.Attention4(layer4)
        
        layer5 = self.layer5(layer4)
        
        global_avg = torch.squeeze(self.global_avg(layer5))
        dropped = self.dropout(global_avg)
        return self.Linear(dropped)

In [ ]:
class MultiFoldQTPi(nn.Module):
    def __init__(self, num_classes, device):
        super().__init__()
        self.num_classes = num_classes
        self.device = device
        self.image_size = 320
        self.stochastic_depth = 0.0
        self.dropout = 0.2
        
        self.model = ModifiedEffNetQT(self.image_size, self.num_classes, self.device, stochastic_depth = self.stochastic_depth, dropout = self.dropout)
    def forward(self, x):
        self.eval()
        with torch.no_grad():
            return torch.sigmoid(self.model(x))

# Load All Models into one for testing

In [ ]:
def load_model():
    '''
    Loads all model
    '''
    mobile = MobileSolverQTPi(len(CLASSES), device)
    if mobile_net_path != "":
        mobile.load_state_dict(torch.load(mobile_net_path, map_location = device))
    mobile.to(device)
    
    eff0 = MultiFoldQTPi(len(CLASSES), device)
    if eff_net_fold_0_path !="":
        eff0.load_state_dict(torch.load(eff_net_fold_0_path, map_location = device))
    eff0.to(device)
    
    eff1 = MultiFoldQTPi(len(CLASSES), device)
    if eff_net_fold_1_path != "":
        eff1.load_state_dict(torch.load(eff_net_fold_1_path, map_location = device))
    eff1.to(device)
    
    eff2 = MultiFoldQTPi(len(CLASSES), device)
    if eff_net_fold_2_path != "":
        eff2.load_state_dict(torch.load(eff_net_fold_2_path, map_location = device))
    eff2.to(device)
    return mobile, eff0, eff1, eff2

In [ ]:
class Tester(nn.Module):
    '''
    Combines all 4 models for inference
    '''
    def __init__(self, mobile, eff0, eff1, eff2):
        super().__init__()
        self.mobile_weight = 0.25
        self.eff0_weight = 1
        self.eff1_weight = 1
        self.eff2_weight = 1
        
        self.mobile = mobile
        self.eff0 = eff0
        self.eff1 = eff1
        self.eff2 = eff2
        
        self.divide = self.mobile_weight + self.eff0_weight + self.eff1_weight + self.eff2_weight
        
    def forward(self, x):
        self.eval()
        with torch.no_grad():
            image, image_vert, image_hor, image_double = self.tta(x)
            pred0 = self.pred_one_model(self.mobile, image, image_vert, image_hor, image_double) * self.mobile_weight
            pred1 = self.pred_one_model(self.eff0, image, image_vert, image_hor, image_double) * self.eff0_weight
            pred2 = self.pred_one_model(self.eff1, image, image_vert, image_hor, image_double) * self.eff1_weight
            pred3 = self.pred_one_model(self.eff2, image, image_vert, image_hor, image_double) * self.eff2_weight
            
            return (pred0 + pred1 + pred2 + pred3) / self.divide
            
    def threshold(self, pred):
        bools = pred >= 0.5
        pred[:, :] = 0
        pred[bools] = 1
        return pred
    def pred_one_model(self, model, image, image_vert, image_hor, image_double):
        pred = model(image)
        pred1 = model(image_vert)
        pred2 = model(image_hor)
        pred3 = model(image_double)
        return (pred + pred1 + pred2 + pred3) / 4 
    def tta(self, image):
        image_vert = image.flip(-2)
        image_hor = image.flip(-1)
        image_double = image.flip(-1).flip(-2)
        return image, image_vert, image_hor, image_double
        

In [ ]:
class PlantInference(nn.Module):
    def __init__(self, CLASSES):
        super().__init__()
        self.classes = CLASSES
        self.num_classes= len(self.classes) 
        
        self.idx2class = {}
        self.class2idx = {}
        for idx in range(self.num_classes):
            self.idx2class[idx] = self.classes[idx]
            self.class2idx[self.classes[idx]] = idx
    def decode(self, idx):
        return self.idx2class[idx]

In [ ]:
def submit(data):
    df = pd.DataFrame(data)
    df = df.set_index('image')
    df.to_csv("./submission.csv", index_label = 'image')
def create_submission(pred):
    '''
    pred: Tensor(C) 
    '''
    inference_config = PlantInference(CLASSES)
    C = pred.shape[0]
    string = ''
    for i in range(C):
        val = pred[i].item()
        if val == 1:
            if string == '':
                string += inference_config.decode(i)
            else:
                string += f' {inference_config.decode(i)}'
    if string == '':
        return 'healthy'
    return string
        
    
def make_submission(model, dataloader):
    data = {'image': [], 'labels': []}
    for images, ids in tqdm.tqdm(dataloader):
        images = images.to(device) 
        ids = list(ids)
        pred = model(images) 
        threshold = model.threshold(pred) 
        B, C = threshold.shape
        for b in range(B):
            one_val = threshold[b]
            data['labels'] += [create_submission(one_val)]
        data['image'] += ids
    submit(data)
    return data
            

In [ ]:
make_submission(Tester(*load_model()), test_dataloader)